# Web Search, Search Engine, and File Search API Journey

Welcome to this interactive tutorial! 🚀

## What You'll Learn

In this notebook, we'll explore how modern AI-powered search works behind the scenes. By the end, you'll understand:

1. **What is AI Overview?** - How Google and other search engines use AI to summarize search results
2. **The Latency Problem** - Why search can be slow and how we can make it faster
3. **Building Your Own Search Engine** - How to create a fast, custom search using vector databases
4. **Connecting AI to Custom Data** - How to make AI answer questions using your own content

## Prerequisites

- **No Python expertise required!** We'll explain each code block step by step
- You'll need an **Upstage API key** (we'll show you where to get one)
- Basic understanding of how web search works (you use Google, right? That's enough!)

## Why This Matters

Imagine you have a collection of articles, documents, or data. Wouldn't it be amazing if an AI could:
- Search through your content instantly (under 1 second!)
- Provide intelligent summaries like Google's AI Overview
- Answer questions based on YOUR specific data

That's exactly what we're building today. Let's get started!

---
# Part 1: Understanding AI Overview
---

## What is Google AI Overview?

Have you noticed that when you search on Google, sometimes you get a nice **summary at the top** instead of just a list of links? That's called **AI Overview** (previously known as "Search Generative Experience" or SGE).

### How It Works:
1. You type a question (e.g., "달리기 잘하는 방법" - "How to run well")
2. Google searches the web for relevant pages
3. An AI reads those pages and writes a **summary** for you
4. You get an instant answer without clicking multiple links!

### Example: AI Overview in Action

![Google AI Overview](./images/google_overview.png)

☝️ **Look at the image above**: When searching "달리기 잘하는 방법" (How to run well), Google's AI provides a comprehensive answer about proper posture, training, and physical conditioning - all synthesized from multiple sources!


## The Problem: AI Overview Doesn't Always Appear

Here's the catch - **Google's AI Overview doesn't work for every search!**

### When Does AI Overview NOT Appear?
- **Niche topics**: Specialized or less common questions
- **Specific company/product queries**: Questions about smaller businesses or specific products
- **Regional content**: Content that's primarily in one language or region
- **New or emerging topics**: Things that haven't been widely covered yet

![No Google AI Overview](./images/no_google_overview.png)

☝️ **In this example**, the search doesn't trigger an AI Overview - you just get regular search results.

### Our Solution: Build Our Own AI Overview! 🛠️

What if we could create our own AI Overview system that:
- Works with **our own content** (articles, documents, databases)
- Provides answers for **any topic we have data on**
- Is **faster** than waiting for Google

**That's exactly what we'll build using Upstage's Solar-Pro2 model!**

---
# Part 2: The Speed Problem
---

## Why Is Search So Slow? ⏱️

When building AI-powered search, we discovered a frustrating problem: **searches were taking 8 seconds!**

### What's Happening Behind the Scenes?

When you ask an AI a question that requires current information:

```
User Question → AI needs fresh data → Calls external search API → Waits... → Gets results → Generates answer
                                              ↑
                                    This step is SLOW! (Tavily: ~1.5-2 seconds)
```

![8 Sec for Search](./images/8_sec_for_search.png)

### Why 8 Seconds is Unacceptable

- **User Experience**: People expect answers in under 2 seconds
- **Competitive Disadvantage**: Google's AI Overview appears almost instantly
- **Multiple Calls**: If you need to search multiple times, delays compound

### The Bottleneck: External Search APIs

Most AI applications use third-party search APIs (like Tavily, Serper, or Bing API) to get fresh web data. These APIs are convenient but **slow** because they:
1. Scrape Google results (SERP scraping)
2. Process and format the data
3. Return results over the network

**We need a better solution!**


## Search API Latency: A Reality Check 📊

Let's look at how fast (or slow) different search APIs actually are. This data comes from real-world measurements:

![perplexity Search](./images/perplexity_search.avif)

### Understanding the Numbers

**What do p50, p75, p90, p95 mean?**
- **p50 (median)**: 50% of requests complete faster than this time
- **p75**: 75% of requests complete faster than this time  
- **p90**: 90% of requests complete faster than this time
- **p95**: 95% of requests complete faster than this time (only 5% are slower)

### Search API Latency Comparison
*(requests initiated from AWS us-east-1)*

| Provider | p50 | p75 | p90 | p95 |
|----------|-----|-----|-----|-----|
| **Perplexity** | 358ms | 443ms | 604ms | 763ms |
| **Exa** | 1375ms | 1499ms | 1798ms | 2188ms |
| **Brave** | 513ms | 637ms | 728ms | 808ms |
| **SERP Based*** | 1342ms | 1586ms | 1743ms | 1790ms |

\* Many search API vendors use SERP scraping as the basis for their results. To capture the performance of such approaches, we use a representative Google SERP-based API offered by Tavily.

### Key Takeaways

1. **Perplexity is fastest** but still ~350-750ms per search
2. **SERP-based APIs (like Tavily) are slowest** at 1.3-1.8 seconds
3. **None of these are fast enough** for a great user experience when combined with LLM processing

**Our goal: Build a search engine with <100ms latency!**

---
# Part 3: Let's Build! Using Solar-Pro2 for AI Overview
---

## Simple Web Search RAG + Solar-Pro2

**RAG** stands for **Retrieval-Augmented Generation**. It's a fancy term for a simple concept:

```
RAG = Search for relevant information + Feed it to an AI + Get an intelligent answer
```

### What is Solar-Pro2?

**Solar-Pro2** is Upstage's powerful language model that can:
- Understand and answer questions in multiple languages (especially Korean!)
- Generate high-quality summaries and explanations
- Work with web search to provide up-to-date answers

### How Solar Web Search Works

1. **You ask a question** → "업스테이지가 1등일수 밖에 없는 이유" (Why Upstage must be #1)
2. **System searches the web** → Finds relevant articles and pages
3. **Solar-Pro2 reads the results** → Understands the content
4. **Returns an AI Overview** → A well-organized answer with citations!

> 💡 **Learn more**: See the prompt engineering behind this at:
> https://github.com/UpstageAI/solar-web-search/blob/main/prompts.py

### Let's Try It! (Code Explanation Below)


### Step 1: Setting Up Our Environment

The code below does the following:

1. **Imports libraries** - Tools we need (like `requests` to call APIs)
2. **Loads API keys** - Your secret key to access Upstage services (stored safely in a `.env` file)
3. **Sets configuration** - Where our search engine lives and what collection to use

> 🔑 **Important**: You'll need to create a `.env` file with your `UPSTAGE_API_KEY`. 
> Get your free API key at: https://console.upstage.ai/


In [7]:
import json
import requests
import time
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
import concurrent.futures
from datetime import datetime, timedelta
import random
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv('.env', override=True)

# Configuration
SEARCH_ENGINE_BASE_URL = os.getenv('SEARCH_ENGINE_BASE_URL', 'https://search_api.toy.x.upstage.ai')
UPSTAGE_API_KEY = os.getenv('UPSTAGE_API_KEY')
COLLECTION_NAME = "sung_test"
BATCH_SIZE = 20
MAX_WORKERS = 1

if not UPSTAGE_API_KEY:
    raise ValueError("UPSTAGE_API_KEY not found in .env.local file")


print(f"Target: {SEARCH_ENGINE_BASE_URL}")
print(f"Collection: {COLLECTION_NAME}")

Target: https://search_api.toy.x.upstage.ai
Collection: sung_test


### Step 2: Calling Solar-Pro2 Search API

Now let's make our first AI Overview request! Here's what the code does:

**The API Endpoint:**
```
https://solar-web-search.cosmic.upstage.ai/v1/chat/completions
```

**The Request Structure:**
- `model`: "solar-pro2-search" - The AI model with built-in web search
- `messages`: Your question (just like chatting!)
- `stream`: False - Wait for the complete response (set to True for real-time streaming)

**What Happens:**
1. We send our question to Solar-Pro2
2. Solar-Pro2 searches the web for relevant information
3. It generates a comprehensive answer with citations [1], [2], etc.
4. We get back a nicely formatted AI Overview!


In [2]:
AI_OVERVIEW_URL = "https://solar-web-search.cosmic.upstage.ai/v1/chat/completions"

payload = {
  "model": "solar-pro2-search",
  "messages": [
    {
      "role": "user",
      "content": "업스테이지가 1등일수 밖에 없는 이유"
    }
  ],
  "stream": False,
}

headers = {
  'Authorization': f'Bearer {UPSTAGE_API_KEY}',
  'Content-Type': 'application/json'
}

response = requests.post(AI_OVERVIEW_URL, headers=headers, json=payload)

print(json.dumps(response.json(), indent=2))


{
  "id": "260c2baf-bf5d-4527-be3c-a76db55ea1e2",
  "object": "chat.completion",
  "created": 1764453087,
  "model": "solar-pro2-search",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "**\uc5c5\uc2a4\ud14c\uc774\uc9c0\ub294 \ud5c8\uae45\ud398\uc774\uc2a4 Open LLM \ub9ac\ub354\ubcf4\ub4dc\uc5d0\uc11c \uc138\uacc4 1\uc704\ub97c \ucc28\uc9c0\ud55c SOLAR \ubaa8\ub378\uacfc \uac15\ub825\ud55c B2B \uc194\ub8e8\uc158 \ud3ec\ud2b8\ud3f4\ub9ac\uc624\ub97c \ubc14\ud0d5\uc73c\ub85c \uae30\uc220\uc801 \uc6b0\uc704\uc640 \uc2e4\uc6a9\uc801\uc778 \uc801\uc6a9 \uac00\ub2a5\uc131\uc744 \uc785\uc99d\ud558\uba70 \uacbd\uc7c1\ub825\uc744 \uc720\uc9c0\ud558\uace0 \uc788\uc2b5\ub2c8\ub2e4 [1][2].**  \n\n### \uae30\uc220\uc801 \uc6b0\uc218\uc131  \n- **SOLAR \ubaa8\ub378\uc758 \uc131\uacfc**: \uc5c5\uc2a4\ud14c\uc774\uc9c0\uc758 \uc790\uccb4 \uac1c\ubc1c LLM\uc778 SOLAR\ub294 \ud5c8\uae45\ud398\uc774\uc2a4 Open LLM \ub9ac\ub354\ubcf4\ub4dc\uc5d0\uc

---
# Part 4: Making It Real - The AskUp Chrome Extension
---

## From Code to Product: AskUp AI Overview 🎉

We turned this technology into a **Chrome browser extension** that anyone can use!

### Performance Results

**First Search: Under 2 Seconds!**

![AskUp Overview](./images/askup_overview.png)

☝️ The AI Overview appears right in your Google search results - no need to visit any website!

---

**With Caching: Just 38 Milliseconds! ⚡**

![AskUp Overview](./images/askup_overview_cache.png)

☝️ When we've seen the question before, we can respond almost instantly using cached results.

### How Caching Works

```
First time asking "What is AI?" 
  → Search + Generate → ~1.5 seconds

Second time asking "What is AI?"
  → Return cached answer → 38ms (40x faster!)
```

---

### Try It Yourself! 🔗

**Install AskUp AI Overview:**
https://chromewebstore.google.com/detail/askup-ai-overview/fdafojjfmhkfdhigkphhgkcahokajcgi 

![AskUp Overview QR](./images/askup_overview_qr.png)

Scan the QR code or click the link to add AI Overview to your Chrome browser!


---
# Part 5: Building a Sub-1-Second Search Engine
---

## The Secret: Vector Databases 🗄️

To achieve **sub-1-second search**, we need to build our own search engine instead of relying on slow external APIs.

### What is a Vector Database?

Traditional databases store data as rows and columns. **Vector databases** store data as **mathematical representations** (vectors) that capture the *meaning* of text.

```
Traditional Search: "smartphone OS" → Exact word match only
Vector Search: "smartphone OS" → Also finds "mobile operating system", "Android", "iOS"
```

This is called **semantic search** - searching by meaning, not just keywords!

### Choosing the Right Vector Database

We evaluated several options. Here's our analysis:

### Table 1: Comparative Analysis of Vector Database Capabilities

| Database | Latency Profile | Payload Handling | Suitability for News | Quantization Features |
|----------|-----------------|------------------|----------------------|-----------------------|
| **Qdrant** ⭐ | Excellent. Written in Rust. High concurrent throughput. Benchmarks show <25ms latency for filtered searches. | Superior. Supports on-disk payloads (mmap) while keeping vectors in RAM. Payload storage is decoupled from vector index. | High. Native support for hybrid search and multi-vectors (ColBERT). Native decay functions for recency. | Advanced. Binary Quantization (BQ) with rescoring allows 40x speedup with minimal accuracy loss. |
| **Redis** | Fastest (Simple). In-memory K/V store. Sub-millisecond for simple lookups. | Poor for Large Text. Storing large text blobs in RAM is expensive. Persistence is complex and not the primary design goal. | Medium. Fast, but lacks advanced filtering flexibility compared to specialized VDBs. | Limited. Less mature quantization options compared to Qdrant. |
| **Weaviate** | Good. Strong hybrid search implementation. Latency can be higher (320-550ms) compared to Qdrant in some scenarios. | Good. Object storage is flexible. Schema enforcement is more rigid. | High. Strong ecosystem and hybrid support, but slightly heavier resource footprint. | Standard. Supports various compression methods. |
| **Milvus** | Moderate. Fast indexing, but higher latency/resource usage for small-scale high-dimension setups. | Robust. Handles large scale well, but operational complexity is higher. | Medium. Hybrid search is supported but can be operationally heavier. | Standard. Supports various compression methods. |

### Our Choice: Qdrant ⭐

We chose **Qdrant** because:
- **Blazing fast**: <25ms for filtered searches
- **Rust-powered**: High performance and reliability
- **Hybrid search**: Combines vector + keyword search for best results
- **Efficient storage**: Can handle large text content without breaking the bank

## Engineering for Sub-Second Latency ⚡

When your goal is **under 1 second**, every millisecond matters! Here's how we budget our time:

### The Search Pipeline (What Happens When You Search)

```
Your Question → [Encode] → [Search Database] → [Fetch Content] → [Rank Results] → [Send Response]
                 ↓            ↓                  ↓                ↓                 ↓
                20-50ms      30-60ms            10-20ms         20-300ms           10ms
```

### Table 3: Latency Budget Breakdown

| Step | Operation | Time Budget | How We Optimize |
|------|-----------|-------------|-----------------|
| 1️⃣ | **Query Encoding** | 20-50ms | Convert your question into a vector using fast embedding models |
| 2️⃣ | **Hybrid Retrieval** | 30-60ms | Qdrant's HNSW Index finds similar content lightning-fast |
| 3️⃣ | **Payload Fetching** | 10-20ms | SSD memory mapping reads content without copying data |
| 4️⃣ | **Reranking** | 20-300ms | Re-sort results by relevance (GPU: 20ms, CPU: 150-300ms) |
| 5️⃣ | **Response Assembly** | 10ms | Package results as JSON |
| | **Total** | **~250-450ms** | **Well under our 1000ms limit!** ✅ |

### Key Optimization Techniques Explained

- **HNSW Index**: A smart data structure that finds similar items without checking every single one
- **Binary Quantization**: Compresses vectors to use less memory and search faster (40x speedup!)
- **Memory Mapping (mmap)**: Reads data directly from disk without copying to RAM first
- **ColBERT Reranking**: A fast neural network that understands context to rank results better

---
# Part 6: Hands-On - Building Your Own Search Engine
---

## Step 1: Loading and Indexing Documents 📚

Now let's build our own search engine! We'll:
1. **Add documents** to our search index
2. **Search** through them
3. **Connect** them to Solar-Pro2 for AI Overview

### What is "Indexing"?

Think of it like building a library catalog:
- **Without index**: To find a book, you'd have to look at every single book
- **With index**: You look up the topic in the catalog and go straight to the right shelf

In our case, indexing means:
1. Taking your documents (articles, news, etc.)
2. Converting them into searchable vectors
3. Storing them in Qdrant for fast retrieval

### Let's Add Some Sample Documents!

### Understanding the Indexing Code

The code below sends documents to our search engine. Let's break it down:

**API Endpoint:** `/index/{collection_name}`
- A "collection" is like a folder for your documents
- You can have multiple collections for different purposes

**Document Structure:**
```json
{
  "title": "Article Title",      // The headline
  "url": "https://...",          // Where it came from
  "content": "Full text...",     // The actual content to search
  "date": "2024-07-20"           // When it was published
}
```

**What happens behind the scenes:**
1. Our API receives the documents
2. Each document is converted into a vector (embedding)
3. The vector + document are stored in Qdrant
4. Ready for lightning-fast search! ⚡


In [3]:
import requests
import json

url = SEARCH_ENGINE_BASE_URL.rstrip('/') + "/index/" + COLLECTION_NAME
print(url)
headers = {
    "Authorization": f"Bearer {UPSTAGE_API_KEY}", # Using the API key from Colab secrets and stripping any extra whitespace/newlines
    "Content-Type": "application/json"
}

data = {
    "documents": [
      {
        "title": "Quantum Computing Breakthrough: New Algorithm Achieves Unprecedented Speed",
        "url": "https://www.techinnovationdaily.com/quantum-breakthrough-speed",
        "content": "Researchers at Quantum Labs have announced a groundbreaking new algorithm that significantly boosts the speed of quantum computations, potentially accelerating the development of new materials and drug discovery. The algorithm, detailed in 'Nature Physics', shows a 10x improvement over previous methods.",
        "date": "2024-07-20"
      },
      {
        "title": "AI Ethics Debate Intensifies as New Facial Recognition Software Raises Privacy Concerns",
        "url": "https://www.digitalfuturegazette.com/ai-ethics-privacy-concerns",
        "content": "A newly released facial recognition software by a prominent tech firm has ignited fresh debates around AI ethics and individual privacy. Critics argue the lack of regulation and potential for misuse poses significant societal risks, while proponents highlight its benefits for public safety.",
        "date": "2024-07-19"
      },
      {
        "title": "Sustainable Energy Sector Sees Record Investment in Q2 2024",
        "url": "https://www.greenpowertoday.com/sustainable-investment-q2",
        "content": "The second quarter of 2024 marked an all-time high for investments in sustainable energy projects globally. Venture capital and private equity firms poured billions into solar, wind, and geothermal technologies, signaling a strong market shift towards renewable resources.",
        "date": "2024-07-18"
      },
      {
        "title": "Major Tech Company Announces Layoffs Amid Restructuring Efforts",
        "url": "https://www.globaltechchronicle.com/company-layoffs-restructuring",
        "content": "One of the world's largest technology companies has announced a significant round of layoffs as part of a broader restructuring initiative. The move is aimed at streamlining operations and refocusing on core product lines, affecting thousands of employees across various departments.",
        "date": "2024-07-17"
      },
      {
        "title": "New Smartphone OS Unveiled: Focus on AI Integration and User Privacy",
        "url": "https://www.mobileworldweekly.com/new-os-ai-privacy",
        "content": "A highly anticipated new smartphone operating system was unveiled today, promising deep AI integration for personalized user experiences alongside enhanced privacy features. Developers are eager to begin building applications for the new platform, which aims to redefine mobile computing.",
        "date": "2024-07-21"
      }
    ]
}

# Adding verify=False due to previous SSL certificate issues
response = requests.post(url, headers=headers, data=json.dumps(data), verify=False)

print(f"Status Code: {response.status_code}")
print(f"Response Body:\n{json.dumps(response.json(), indent=2)}")

https://search_api.toy.x.upstage.ai/index/themilk
Status Code: 200
Response Body:
{
  "status": "success",
  "count": 5
}


## Step 2: Testing Our Search Engine 🔍

Now that we've indexed documents, let's search through them!

### How Search Works

```
Your Question: "What's new smartphone OS?"
       ↓
[Convert to vector] → [Find similar vectors in Qdrant] → [Return matching documents]
       ↓
Results: Articles about smartphone operating systems, AI integration, privacy features...
```

### Understanding the Search Response

The search returns a list of **results**, each containing:
- `title`: The article headline
- `snippet`: A short preview of the content
- `content`: The full text
- `score`: How relevant it is (1.0 = perfect match)
- `url`: Source link
- `date`: Publication date

**Let's try searching for "What's new smartphone OS?":**

In [6]:
import requests
import json

search_endpoint = SEARCH_ENGINE_BASE_URL.rstrip('/') + "/search/" + COLLECTION_NAME
print(url)
headers = {
    "Authorization": f"Bearer {UPSTAGE_API_KEY}", # Using the api_key obtained previously
    "Content-Type": "application/json"
}

data = {
    "query": "What's new smart phone OS?",
    "limit": 5
}

# Adding verify=False due to previous SSL certificate issues
response = requests.post(search_endpoint, headers=headers, data=json.dumps(data), verify=False)

print(f"Status Code: {response.status_code}")
print(f"Response Body: {response.json()}")
# --- END MODIFIED CODE ---

https://search_api.toy.x.upstage.ai/search/themilk
Status Code: 200
Response Body: {'results': [{'title': 'New Smartphone OS Unveiled: Focus on AI Integration and User Privacy', 'url': 'https://www.mobileworldweekly.com/new-os-ai-privacy', 'snippet': 'A highly anticipated new smartphone operating system was unveiled today, promising deep AI integration for personalized user experiences alongside enhanced privacy features. Developers are eager to begin building applications for the new platform, which aims to redefine mobile computing.', 'content': 'A highly anticipated new smartphone operating system was unveiled today, promising deep AI integration for personalized user experiences alongside enhanced privacy features. Developers are eager to begin building applications for the new platform, which aims to redefine mobile computing.', 'date': '2024-07-21', 'last_updated': None, 'full_content': 'A highly anticipated new smartphone operating system was unveiled today, promising deep AI in

---
# Part 7: The Magic - AI Overview with YOUR Data
---

## Step 3: Connecting Solar-Pro2 to Our Search Engine 🪄

This is where everything comes together! We'll connect Solar-Pro2 to use **our own search engine** instead of the public web.

### The Key Configuration

```python
"web_search_options": {
    "external_search_engine_only": True,  # Use ONLY our search engine
    "servers": [{
        "url": "our_search_endpoint",     # Point to our Qdrant-based search
        "key": "our_api_key"
    }]
}
```

### What This Enables

1. **Privacy**: Your data never leaves your control
2. **Customization**: AI answers based on YOUR content
3. **Speed**: Sub-second responses (no slow external APIs!)
4. **Accuracy**: Answers grounded in your specific domain knowledge

### Let's Try It!

We'll ask about "윤송이 박사님 강의" (Dr. Yoon's lecture) using our indexed content:

In [10]:
AI_OVERVIEW_URL = "https://solar-web-search.cosmic.upstage.ai/v1/chat/completions"

payload = {
  "model": "solar-pro2-search",
  "messages": [
    {
      "role": "user",
      "content": "윤송이 박사님 강의"
    }
  ],
  "stream": False,
  "web_search_options": {
    "search_context_size": "medium",
    "external_search_engine_only": True,
    "servers": [
      {
        "url": search_endpoint,
        "key": UPSTAGE_API_KEY
      }
    ]
  }
}

headers = {
  'Authorization': f'Bearer {UPSTAGE_API_KEY}',
  'Content-Type': 'application/json'
}

print(f"Querying Solar API with external search: {search_endpoint}...")
response = requests.post(AI_OVERVIEW_URL, headers=headers, json=payload)

print(json.dumps(response.json(), indent=2))


Querying Solar API with external search: https://search_api.toy.x.upstage.ai/search/themilk...
{
  "id": "6d3552e5-536c-4dcf-8902-62e1b2c7ddf3",
  "object": "chat.completion",
  "created": 1764422960,
  "model": "solar-pro2-search",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "**\uc724\uc1a1\uc774 \ubc15\uc0ac\ub294 AI\ub97c \uc778\ub958 \uc9c4\ud654\uc758 \ubcc0\uace1\uc810\uc73c\ub85c \ubcf4\uba70, \ubd88\uacfc \uc5b8\uc5b4\uc5d0 \uc774\uc740 \uc138 \ubc88\uc9f8 \uadfc\uc6d0\uc801 \ubcc0\ud654 \ub3c4\uad6c\ub85c \uac15\uc870\ud588\uc2b5\ub2c8\ub2e4 [1][4][8].**  \n\n### \uc0c1\uc138 \uc124\uba85  \n1. **AI\ub97c \uc778\ub958 \ubb38\uba85\uc758 \uc138 \ubc88\uc9f8 \ubd88\uae38\ub85c \uaddc\uc815**  \n   - \uc724\uc1a1\uc774 PVP \ub300\ud45c\ub294 2026 \uae30\uc870\uc5f0\uc124\uc5d0\uc11c \"AI\ub294 \ub300\ud55c\ubbfc\uad6d\uc758 \uc138 \ubc88\uc9f8 \ubd88\"\uc774\ub77c\uace0 \uc8fc\uc7a5\ud558\uba70, \ubd88\uacfc \uc5b8\uc5

---
# Part 8: Real-World Application - The Milk 🥛
---

## Case Study: AI-Powered News Search for The Milk

**The Milk** (더밀크) is a Korean business/tech news platform. We've integrated our search engine to power their AI features!

### The Search Experience

![Themilk search](./images/themilk_search.png)

☝️ Users can ask questions about business news and get AI-generated answers based on The Milk's article database - exactly like Google's AI Overview, but for their specific content!

---

### Business Impact

![Themilk sales](./images/themilk_sales.png)

This technology enables:
- **Better user engagement**: Users find answers faster
- **Content monetization**: AI answers drive subscriptions
- **Competitive advantage**: Features that major platforms don't offer for niche content

---

## 🎉 Congratulations!

You've learned how to:
1. ✅ Understand what AI Overview is and why it matters
2. ✅ Identify the latency problems with external search APIs
3. ✅ Choose the right vector database (Qdrant!)
4. ✅ Build and index a document collection
5. ✅ Search through documents with semantic search
6. ✅ Connect Solar-Pro2 to your own search engine
7. ✅ Create your own AI Overview experience!

### Next Steps

- 🔗 Get your API key: https://console.upstage.ai/
- 📚 Explore Solar-Pro2: https://developers.upstage.ai/
- 🧩 Try the AskUp Extension: Install it from the Chrome Web Store!

**Happy building! 🚀**
